In [1]:
# Junta todos os arquivos de resultado do map matching

# carregar bibliotecas
source('fun/setup.R')

# Dados a atualizar de acordo com o mês a rodar
ano_mes <- '201811'
ano_mes <- '201812'
ano_mes <- '201901'

# Estrutura de pastas
pasta_dados        <- "../../yellow_dados"
pasta_map_matching <- sprintf("%s/05_map_matching", pasta_dados)
pasta_mes_mapmatch <- sprintf("%s/%s", pasta_map_matching, ano_mes)
# pasta_viagens_csv <- sprintf("%s/viagens_processadas_csv", pasta_mes_mapmatch)
pasta_viagens_log <- sprintf("%s/viagens_processadas_log", pasta_mes_mapmatch)

GDAL version >= 3.1.0 | setting mapviewOptions(fgb = TRUE)

Linking to GEOS 3.9.1, GDAL 3.2.2, PROJ 8.0.0


Attaching package: ‘jsonlite’


The following object is masked from ‘package:purrr’:

    flatten




In [2]:
# ----------------------------------------------------------
# Juntar e salvar resultados em arquivo único
# ----------------------------------------------------------

# Pegar nomes de todos os .csvs de resultado
result_files <- list.files(pasta_viagens_log, pattern = '*.csv', full.names = TRUE)

# Criar um dataframe único para todos os registros daquele dia
# https://stackoverflow.com/questions/46299777/add-filename-column-to-table-as-multiple-files-are-read-and-bound
result_files <- data.frame(n_seq = result_files)

# Dataframe temporário para processar uma certa quantidade de linhas
tmp_dia <-
  result_files %>%
  # Abrir cada arquivo e extrair conteúdo em nova coluna
  mutate(reg = lapply(n_seq, read_delim, delim = ';', col_types = cols(.default = "c"))) %>% 
  # Transformar cada linha de conteúdo em uma linha nova no tibble (linhas com
  # nomes de arquivos vão se repetir para cada linha de conteúdo)
  unnest(reg, keep_empty = TRUE) %>% 
  # Pegar a primeira linha e usar como nomes de colunas, depois descartá-la
  # setNames(slice(., 1)) %>% 
  # Nomes das colunas vieram junto - retirar essas linhas (todas repetidas)
  # filter(trip_id != 'trip_id') %>% 
  # Retirar primeira coluna, com referência aos endereços do arquivo
  select(2:12)

mutate: new variable 'reg' (list) with 58,510 unique values and 0% NA

slice: removed 117,019 rows (>99%), one row remaining

filter: removed 58,510 rows (50%), 58,510 rows remaining

select: dropped one variable (../../yellow_dados/05_map_matching/201901/viagens_processadas_log/000265_00.csv)



In [3]:
tmp_dia %>% head()

trip_id,cod_proc,n_points,qtd_quebras,qtd_it_outliers,prop_centr_100,tempo_total,dist_total,veloc_vg_kph,vg_inicio,vg_termino
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
000265,00101000000,70,0,2,32.86,359.29299998283386,1082.006919173991,10.841360419525213,2019-01-02T08:10:14Z,2019-01-02T08:16:14Z
000266,00001000000,212,0,2,11.79,1109.667000055313,1632.1215917486948,5.294955811069826,2019-01-04T06:11:14Z,2019-01-04T06:29:43Z
000268,00101000000,145,0,3,0.69,1219.4969999790192,4217.373076382189,12.449840446706377,2019-01-04T20:21:07Z,2019-01-04T20:41:27Z
000271,01111100000,463,2,3,0.22,3100.684000015259,6402.112994938577,7.433071793728564,2019-01-05T18:18:38Z,2019-01-05T19:10:18Z
000274,00101000000,198,0,2,0,2023.816999912262,6323.225945048893,11.247861542403726,2019-01-07T18:08:11Z,2019-01-07T18:41:55Z
000275,00101000000,80,0,2,2.5,920.9500000476837,2432.9864241339915,9.510560971202421,2019-01-07T19:10:20Z,2019-01-07T19:25:41Z


In [4]:
# Definir arquivo de saída
out_file <- sprintf('%s/%s/%s_resultados_map_matching_rotas_completas.csv', pasta_map_matching, ano_mes, ano_mes)
write_delim(tmp_dia, out_file, delim = ';')